
# Source Code

accompanying the paper: 

**Limited Capability of MRI Radiomics to Predict Primary Tumor Histology of Brain Metastases in External Validation**  

Neuro-Oncology Advances 2024  

DOI: 10.1093/noajnl/vdae060   

## Prerequisites

### Naming Convention: 
_0000: FLAIR  
_0001: T1  
_0002: T1CE     

### Segmentation Labels:    
0: background    
1: edema    
2: necrotic tissue    
3: enhancing tumor   

### Data Structure:
```bash
/BrainMetDataset/  
├── data.xlsx (containing case IDs)    
├── Pat01    
│   ├── Pat01_0000.nii.gz (FLAIR)  
│   ├── Pat01_0001.nii.gz (T1)  
│   ├── Pat01_0002.nii.gz (T1CE)  
├── Pat02  
│   ├── Pat02_0000.nii.gz (FLAIR)  
│   ├── Pat02_0001.nii.gz (T1)  
│   ├── Pat02_0002.nii.gz (T1CE)  
├── Pat03  
│   ├── ...  
```

### Requirements
- SimpleITK
- numpy
- nipype
- fsl
- ants
- HD-BET
- connected-components-3d (cc3d)
- sklearn
- imblearn
- tqdm
- matplotlib
- seaborn
- intensity_normalization
- multiprocessing
- pandas 
- nnunetv1
- scipy
- pyradiomics

## Data Preprocessing

Preprocessing functions can be found under brainmet.utils.preprocessing

Steps:
1. DICOM to NIFTI conversion: dcm2niix_folder
2. Brain Extraction: extract_brain
3. Cropping: get_bounding_box, apply_bounding_box
4. Bias Correction: apply_bias_correction
5. Co-Registration: coregister_nipype or coregister_antspy
6. Intensity Normalization: zscore_normalize

## Tumor Segmentation
- Using a pretrained nnU-Net, e.g., Task001_BrainTumour (trained on glioma data, but also works for brain metastases)
- **CAVE: Segmentations need to be manually verified and corrected!**
- Tumor segmentation before N4Bias-Correction and Normalization may give better results with the pretrained nnU-Net

In [ ]:
from nnunet.inference import predict
import os
from brainmet.utils.config import DATASET_PATH

os.makedirs(os.path.join(DATASET_PATH, "segmented"), exist_ok=True)

predict.predict_from_folder(
    input_folder=os.path.join(DATASET_PATH, "preprocessed/"),
    output_folder=os.path.join(DATASET_PATH, "segmented/"),
    part_id=0,
    num_parts=1,
    model="/PATH/TO/NNUNET/nnUNet_trained_models/nnUNet/3d_fullres/Task001_BrainTumour/nnUNetTrainerV2__nnUNetPlansv2.1/",
    folds="all",
    save_npz=False,
    lowres_segmentations=None,
    num_threads_preprocessing=2,
    num_threads_nifti_save=2,
    tta=True,
    step_size=0.5,
)

## Splitting Tumor Masks
- We will only use the tumor core (necrotic and enhancing tumor)

In [ ]:
!python -m pip install connected-components-3d

In [ ]:
import cc3d
import numpy as np
import SimpleITK as sitk
import pandas as pd
from tqdm import tqdm
from brainmet.utils.preprocessing import binarize_itkimage
from brainmet.utils.config import DATASET_PATH


cases = ["LIST", "OF", "CASE_IDs"]
met_numbers = []

for c in tqdm(cases):
    met_number = {}

    img = sitk.ReadImage(
        os.path.join(DATASET_PATH, c, f"{c}_NAME_OF_SEGMENTATION.nii.gz")
    )
    img_bin = binarize_itkimage(img, l_thresh=2, u_thresh=3)

    arr = sitk.GetArrayFromImage(img_bin).astype(int)

    labels_out, n = cc3d.connected_components(arr, return_N=True, connectivity=26)

    for label, image in cc3d.each(labels_out, binary=False, in_place=True):
        image = np.where(image == label, 1, 0)
        image = image.astype(np.uint8)
        img_new = sitk.GetImageFromArray(image)
        img_new.CopyInformation(img)
        # Each metastasis is saved in a separate numbered label image
        sitk.WriteImage(
            img_new,
            os.path.join(DATASET_PATH, c, f"{c}_MASK_NAME_NUMBER_{label}.nii.gz"),
        )

    met_number["ID"] = c
    met_number["number"] = n
    met_numbers.append(met_number)

met_numbers_df = pd.DataFrame(met_numbers)
met_numbers_df.to_excel(os.path.join(DATASET_PATH, "met_numbers.xlsx"))

## Feature Extraction

In [ ]:
import os
import SimpleITK as sitk
import radiomics
import pandas as pd
import multiprocessing
from tqdm.contrib.concurrent import process_map
from collections import OrderedDict
from brainmet.utils.config import DATASET_PATH

N_PROC = multiprocessing.cpu_count() - 1

# List that contains the number of metastases for each case
number = pd.read_excel(
    os.path.join(DATASET_PATH, "met_numbers.xlsx"), header=0, index_col=None
)
sequences = [
    ("FLAIR", "0001"),
    ("T1SE", "0002"),
    ("T1CE", "0003"),
]

radiomics.setVerbosity(60)

loopList = zip(number.ID.to_list(), number.number.to_list())

params = "Params.yaml"

manager = multiprocessing.Manager()
failed = manager.list()

In [ ]:
def get_features(x):
    """
    Function to extract radiomic features for each case separately and
    store in case folder

    Args:
    Tuple: (case_id, number_of_metastases)
    """

    p = x[0]
    i = x[1]

    global failed

    result_list = []
    images = []

    # Load all structural images for case
    for sequenceName, sequenceNumber in sequences:
        itkimage = sitk.ReadImage(
            os.path.join(DATASET_PATH, p, f"{p}_IMAGE_NAME_{sequenceNumber}.nii.gz")
        )
        images.append((itkimage, sequenceName))

    # Extract features for each metastasis separately
    for j in range(1, int(i) + 1):
        try:
            res = OrderedDict()
            res["ID"] = f"{p}_{j}"

            maskImage = sitk.ReadImage(
                os.path.join(DATASET_PATH, p, f"{p}_MASK_NAME_NUMBER_{j}.nii.gz")
            )

            for img, name in images:
                extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(params)
                result = extractor.execute(img, maskImage, label=1)

                # Add Sequence name as prefix to each Key
                result = {f"{name}_{k}": v for k, v in result.items()}

                # Remove all diagnostics keys
                result = {k: v for k, v in result.items() if not "diagnostics" in k}

                res.update(result)

        except Exception as e:
            failed.append(f"{p}_{j}: {str(e)}")

        result_list.append(res)

    result_table = pd.DataFrame(result_list)
    # keep each shape feature only once
    result_table.columns = result_table.columns.str.replace(".*shape", "shape")
    result_table = result_table.loc[:, ~result_table.columns.duplicated()]
    result_table.to_excel(os.path.join(DATASET_PATH, p, "features.xlsx"), index=False)


if __name__ == "__main__":
    r = process_map(get_features, loopList, max_workers=N_PROC)

    # Save log of instances where extraction failed
    with open(os.path.join(DATASET_PATH, "Feature_extraction_log.txt"), "w") as file:
        for row in failed:
            s = "".join(map(str, row))
            file.write(s + "\n")

In [ ]:
finalResults = pd.DataFrame()

for p in tqdm(number.ID.to_list()):
    data = pd.read_excel(
        os.path.join(DATASET_PATH, p, "features.xlsx"), header=0, index_col=0
    )
    finalResults = pd.concat([finalResults, data], axis=0)


finalResults.dropna(inplace=True)
finalResults.sort_index(axis=1, inplace=True)
finalResults.sort_index(axis=0, inplace=True)

finalResults.to_excel(os.path.join(DATASET_PATH, "features_complete.xlsx"))
print(finalResults.shape)

## Calculate Location of Metastases

In [ ]:
import numpy as np
import SimpleITK as sitk
import pandas as pd
from scipy import ndimage
from tqdm import tqdm

loopList = zip(number.ID.to_list(), number.number.to_list())

data = []

for p, i in tqdm(loopList):
    for j in range(1, int(i) + 1):
        maskName = os.path.join(DATASET_PATH, p, f"{p}_MASK_NAME_NUMBER_{j}.nii.gz")
        itk_image = sitk.ReadImage(maskName)

        image_array = sitk.GetArrayFromImage(itk_image)

        # in z,y,x direction
        CM = ndimage.measurements.center_of_mass(np.array(image_array))

        relativeCOM = np.array(CM) / np.array(image_array.shape)

        data.append([f"{p}_{j}", relativeCOM[2], relativeCOM[1], relativeCOM[0]])

result_table = pd.DataFrame(data, columns=["ID", "x", "y", "z"])

result_table.to_excel(os.path.join(DATASET_PATH, "center_of_mass.xlsx"))

In [ ]:
features = pd.read_excel(
    os.path.join(DATASET_PATH, "features_complete.xlsx"), header=0, index_col=None
)
location = pd.read_excel(
    os.path.join(DATASET_PATH, "center_of_mass.xlsx"), header=0, index_col=None
)
features.set_index("ID", inplace=True)
location.set_index("ID", inplace=True)

features.loc[
    features.index.intersection(location.index), ["x", "y", "z"]
] = location.loc[features.index.intersection(location.index), ["x", "y", "z"]]

features.to_excel(os.path.join(DATASET_PATH, "features_location_complete.xlsx"))

## Machine Learning Classifier Selection

### Prerequisites

In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    log_loss,
)
from sklearn.model_selection import (
    GridSearchCV,
    cross_validate,
    StratifiedGroupKFold,
    permutation_test_score,
)
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.decomposition import KernelPCA
import os
import random
from brainmet.utils.feature_selection import MRMRFeatureSelector, LASSO_selection
from brainmet.utils.config import RESULT_PATH, DATASET_PATH
from brainmet.utils.plotting import (
    plot_distribution,
    plot_heatmap,
    plot_multiclass_roc,
    plot_pca,
)
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, NuSVC
import math
from sklearn.utils import resample


plt.style.use("ggplot")

plt.rcParams.update(
    {
        "text.color": "black",
        "axes.labelcolor": "black",
        "font.weight": "normal",
        "font.size": 16,
        "xtick.color": "black",
        "ytick.color": "black",
    }
)

RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

selected_classes = [
    "Breast Cancer",
    "Colorectal Cancer",
    "Lung Cancer",
    "Melanoma",
    "Other",
]

### Load Tables
- Tables contain radiomic and location features for each metastasis and a column 'Primary' containing the histology
- Index = caseID (to group all metastases of a patient to either train or test set)

In [ ]:
# Train Data
data_train = pd.read_excel(
    os.path.join(DATASET_PATH, "features_with_location_primary_train.xlsx"),
    header=0,
    index_col=0,
    dtype={"Primary": str},
)

# Test Data
data_test_1 = pd.read_excel(
    os.path.join(DATASET_PATH, "features_with_location_primary_test1.xlsx"),
    header=0,
    index_col=0,
    dtype={"Primary": str},
)

data_test_2 = pd.read_excel(
    os.path.join(DATASET_PATH, "features_with_location_primary_test2.xlsx"),
    header=0,
    index_col=0,
    dtype={"Primary": str},
)

### Pipeline Selection

In [ ]:
results = []

X_train = data_train.drop("Primary", axis=1, inplace=False).astype(float)
Y_train = data_train["Primary"]

# Group all metastases of a patient into either train or test set
train_inds, test_inds = next(
    StratifiedGroupKFold(n_splits=5).split(X_train, Y_train, groups=X_train.index)
)

X_train_int = X_train.iloc[train_inds]
X_test_int = X_train.iloc[test_inds]
Y_train_int = Y_train.iloc[train_inds]
Y_test_int = Y_train.iloc[test_inds]

norm_options = [StandardScaler(), MinMaxScaler()]

# with l1 (=LASSO) regularization
lasso_fs = SelectFromModel(
    LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000),
    max_features=5,
)

feature_selection_options = [
    MRMRFeatureSelector(number=5),
    SelectKBest(k=5),
    lasso_fs,
]

oversampling_methods = [
    None,
    SMOTE(random_state=RANDOM_SEED, k_neighbors=4),
    RandomOverSampler(random_state=RANDOM_SEED),
]

classifiers = [
    LogisticRegression(),
    RandomForestClassifier(),
    SVC(probability=True),
    NuSVC(nu=0.01, probability=True),
    AdaBoostClassifier(),
    KNeighborsClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    MLPClassifier(),
]

scoring = {
    "acc": "accuracy",
    "roc_auc": "roc_auc_ovo",
    "f1": "f1_macro",
}

for norm in norm_options:
    for fs in feature_selection_options:
        for ovs in oversampling_methods:
            for clf in classifiers:
                steps = []

                steps.append(("scaler", norm))
                steps.append(("selector", fs))
                if os:
                    steps.append(("oversampler", ovs))
                steps.append(("classifier", clf))
                # Using imbpipeline to ensure only on training data is oversampled
                pipe = imbpipeline(steps)

                scores = cross_validate(
                    pipe,
                    X_train_int,
                    Y_train_int,
                    scoring=scoring,
                    cv=StratifiedGroupKFold(n_splits=5),
                    groups=X_train_int.index,
                    return_train_score=True,
                )

                results.append(
                    {
                        "normalization": norm.__class__.__name__,
                        "feature_selection": fs.__class__.__name__,
                        "oversampling": ovs.__class__.__name__ if os else "None",
                        "classifier": clf.__class__.__name__,
                        "accuracy": round(scores["test_acc"].mean(), 3),
                        "ROC_AUC_macro": round(scores["test_roc_auc"].mean(), 3),
                        "F1_macro": round(scores["test_f1"].mean(), 3),
                    }
                )

results_df = pd.DataFrame(results)
results_df.to_excel(os.path.join(RESULT_PATH, "results_select_classifier.xlsx"))

### Forward Sequential Feature Selection 

In [ ]:
results = []

X_train = data_train.drop("Primary", axis=1, inplace=False).astype(float)
Y_train = data_train["Primary"]

# Group all metastases of a patient into either train or test set
train_inds, test_inds = next(
    StratifiedGroupKFold(n_splits=5).split(X_train, Y_train, groups=X_train.index)
)

X_train_int = X_train.iloc[train_inds]
X_test_int = X_train.iloc[test_inds]
Y_train_int = Y_train.iloc[train_inds]
Y_test_int = Y_train.iloc[test_inds]

oversampling_methods = [
    None,
    SMOTE(random_state=RANDOM_SEED, k_neighbors=4),
    RandomOverSampler(random_state=RANDOM_SEED),
]

scoring = {
    "acc": "accuracy",
    "roc_auc": "roc_auc_ovo",
    "f1": "f1_macro",
}

for i in range(1, math.ceil(0.01 * len(X_train_int.columns))):
    for ovs in oversampling_methods:
        steps = []

        lasso_fs = SelectFromModel(
            LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000),
            max_features=i,
        )

        steps.append(("scaler", StandardScaler()))
        steps.append(("selector", lasso_fs))
        if os:
            steps.append(("oversampler", ovs))
        steps.append(("classifier", RandomForestClassifier(random_state=RANDOM_SEED)))
        pipe = imbpipeline(steps)

        scores = cross_validate(
            pipe,
            X_train_int,
            Y_train_int,
            scoring=scoring,
            cv=StratifiedGroupKFold(n_splits=5),
            groups=X_train_int.index,
            return_train_score=True,
        )

        results.append(
            {
                "Feature_Selection": "Lasso",
                "n_features": i,
                "oversampling": ovs.__class__.__name__ if os else "None",
                "classifier": "Random_Forest",
                "accuracy": round(scores["test_acc"].mean(), 3),
                "ROC_AUC_macro": round(scores["test_roc_auc"].mean(), 3),
                "F1_macro": round(scores["test_f1"].mean(), 3),
            }
        )

results_df = pd.DataFrame(results)
results_df.to_excel(
    os.path.join(RESULT_PATH, "results_sequential_feature_selection.xlsx")
)

### Final Model Training
- all dataset combinations

In [ ]:
dataset_combinations = [
    (
        "Train = Local, Test = STANFORD+USCF",
        data_train,
        pd.concat([data_test_1, data_test_2]),
    ),
    (
        "Train = STANFORD, Test = Local+USCF",
        data_test_1,
        pd.concat([data_train, data_test_2]),
    ),
    (
        "Train = UCSF, Test = Local+STANFORD",
        data_test_2,
        pd.concat([data_train, data_test_1]),
    ),
    (
        "Train = Local+STANFORD, Test = UCSF",
        pd.concat([data_train, data_test_1]),
        data_test_2,
    ),
    (
        "Train = Local+UCSF, Test = STANFORD",
        pd.concat([data_train, data_test_2]),
        data_test_1,
    ),
    (
        "Train = STANFORD+UCSF, Test = Local",
        pd.concat([data_test_1, data_test_2]),
        data_train,
    ),
    (
        "Train = Local, Test = STANFORD",
        data_train,
        data_test_1,
    ),
    (
        "Train = Local, Test = USCF",
        data_train,
        data_test_2,
    ),
    (
        "Train = STANFORD, Test = Local",
        data_test_1,
        data_train,
    ),
    (
        "Train = STANFORD, Test = UCSF",
        data_test_1,
        data_test_2,
    ),
    (
        "Train = UCSF, Test = Local",
        data_test_2,
        data_train,
    ),
    (
        "Train = UCSF, Test = Stanford",
        data_test_2,
        data_test_1,
    ),
]

#### "Correct" Approach

In [ ]:
results = []
N_FEATURES = "NUMBER_OF_FEATURES_TO_SELECT"

for combination_name, train_data, test_data in dataset_combinations:
    # select columns for X and Y
    X_train = train_data.drop("Primary", axis=1, inplace=False).astype(float)
    Y_train = train_data["Primary"]

    X_test = test_data.drop("Primary", axis=1, inplace=False).astype(float)
    Y_test = test_data["Primary"]

    # Create Train/Test Split for Train Data, grouping metastases
    # of the same patient in either train or test set
    train_inds, test_inds = next(
        StratifiedGroupKFold(n_splits=5).split(X_train, Y_train, groups=X_train.index)
    )

    X_train_int = X_train.iloc[train_inds]
    X_test_int = X_train.iloc[test_inds]
    Y_train_int = Y_train.iloc[train_inds]
    Y_test_int = Y_train.iloc[test_inds]

    X_test_ext = X_test.copy()
    Y_test_ext = Y_test.copy()

    # Feature Normalization
    # fit only on train data, apply transform to test data
    scaler = StandardScaler()
    X_train_int = pd.DataFrame(
        scaler.fit_transform(X_train_int), columns=X_train_int.columns
    )
    X_test_int = pd.DataFrame(scaler.transform(X_test_int), columns=X_test_int.columns)
    X_test_ext = pd.DataFrame(scaler.transform(X_test_ext), columns=X_test_ext.columns)

    # Select features based on training data
    features = LASSO_selection(X_train_int, Y_train_int, N_FEATURES)
    X_train_int = X_train_int.loc[:, features]
    X_test_int = X_test_int.loc[:, features]
    X_test_ext = X_test_ext.loc[:, features]

    # Oversampling
    SMOTE_Sampler = SMOTE(random_state=RANDOM_SEED, k_neighbors=4)
    X_train_int_SMOTE, Y_train_int_SMOTE = SMOTE_Sampler.fit_resample(
        X_train_int, Y_train_int
    )

    ROS = RandomOverSampler(random_state=RANDOM_SEED)
    X_train_int_ROS, Y_train_int_ROS = ROS.fit_resample(X_train_int, Y_train_int)

    dataframes = [
        (X_train_int, Y_train_int, "Baseline"),
        (X_train_int_SMOTE, Y_train_int_SMOTE, "SMOTE Correct Oversampling"),
        (X_train_int_ROS, Y_train_int_ROS, "ROS Correct Oversampling"),
    ]

    # Plot test set distributions
    plot_distribution(
        Y_test_int, "Internal Test Set Correct Oversampling", combination_name
    )

    plot_distribution(
        Y_test_ext, "External Test Set Correct Oversampling", combination_name
    )

    result = {}
    result["Combination_Name"] = combination_name

    for X, Y, name in dataframes:
        # Plot Label Distribution for Train Set
        plot_distribution(Y, f"Internal Train Set {name}", combination_name)

        print(Y.value_counts(sort=True, normalize=False))

        plot_pca(
            X,
            Y,
            selected_classes,
            f"Internal Train Set {name}",
            combination_name,
            kernel_type="rbf",
            gamma_type=1,
            alpha_type=0.1,
        )

        # Model selection with Cross-Validation
        cv_inner = StratifiedGroupKFold(n_splits=5)
        model = RandomForestClassifier(random_state=RANDOM_SEED)
        space = {
            "n_estimators": [10, 100, 500, 1000],
            "max_features": ["sqrt", "auto", 2, 4, 6],
        }

        search = GridSearchCV(model, space, scoring="f1_macro", cv=cv_inner, refit=True)

        res = search.fit(X, Y, groups=X.index)
        # get the best performing model fit on the whole training set
        best_model = res.best_estimator_
        best_model.fit(X, Y)

        # Calculate permutation score to test vs. random labels
        score, perm_scores, pvalue = permutation_test_score(
            best_model, X, Y, n_permutations=100, n_jobs=4, scoring="f1_macro"
        )

        # Feature Importance
        sorted_idx = best_model.feature_importances_.argsort()
        plt.barh(
            X.columns[sorted_idx],
            best_model.feature_importances_[sorted_idx],
            color="#083471",
        )
        plt.xlabel(f"{name} Feature Importance")
        plt.savefig(
            os.path.join(
                RESULT_PATH, f"Feature_Importance_{combination_name}_{name}.tiff"
            ),
            dpi=300,
            bbox_inches="tight",
        )
        plt.show()
        plt.close()

        predictions_int = pd.Series(
            pd.Categorical(best_model.predict(X_test_int), categories=selected_classes)
        )
        probabilities_int = best_model.predict_proba(X_test_int)
        ytest_dummies_int = pd.get_dummies(Y_test_int, drop_first=False).to_numpy()
        Y_test_int = pd.Series(
            pd.Categorical(Y_test_int, categories=selected_classes),
            index=Y_test_int.index,
        )

        conf_matrix_int = confusion_matrix(
            Y_test_int, predictions_int, labels=selected_classes
        )
        array_int = np.array(conf_matrix_int)

        plot_heatmap(
            normalized_array=array_int,
            classes=best_model.classes_,
            conf_matrix=conf_matrix_int,
            name=f"{combination_name} Internal Test Set {name}",
            Y=Y_test_int,
            combination=combination_name,
        )

        plot_multiclass_roc(
            Y_test_int,
            probabilities_int,
            n_classes=len(selected_classes),
            figsize=(6, 5),
            name=f"{combination_name} Internal Test Set {name}",
            classes=best_model.classes_,
            combination=combination_name,
        )
        result[f"{name}_Length_Train"] = np.shape(Y)[0]
        result[f"{name}_Length_Test_int"] = np.shape(Y_test_int)[0]
        result[f"{name}_Length_Test_ext"] = np.shape(Y_test_ext)[0]
        result[f"{name}_Cross_Validation_Score"] = np.round(search.best_score_, 2)
        result[f"{name}_Permutation_pvalue"] = np.round(pvalue, 4)
        result[f"{name}_Hyperparameters"] = search.best_params_
        result[f"{name}_Features_Number"] = len(features)
        result[f"{name}_Features"] = features
        result[f"{name}_Model"] = "Random Forest Classifier"
        result[f"{name}_Pred_Test_int"] = predictions_int
        result[f"{name}_Proba_Test_int"] = probabilities_int
        result[f"{name}_ACC_int"] = np.round(
            accuracy_score(Y_test_int, predictions_int), 2
        )
        result[f"{name}_F1_int"] = np.round(
            f1_score(Y_test_int, predictions_int, average="macro"), 2
        )
        result[f"{name}_PREC_int"] = np.round(
            precision_score(Y_test_int, predictions_int, average="macro"), 2
        )
        result[f"{name}_REC_int"] = np.round(
            recall_score(Y_test_int, predictions_int, average="macro"), 2
        )
        result[f"{name}_AUC_macro_ovo_int"] = np.round(
            roc_auc_score(
                Y_test_int,
                probabilities_int[:, 1],
                average="macro",
                multi_class="ovo",
                labels=sorted(selected_classes),
            ),
            2,
        )
        result[f"{name}_log_loss_int"] = log_loss(
            Y_test_int, probabilities_int, labels=sorted(selected_classes)
        )

        predictions_ext = best_model.predict(X_test_ext)
        probabilities_ext = best_model.predict_proba(X_test_ext)
        ytest_dummies_ext = pd.get_dummies(Y_test_ext, drop_first=False).to_numpy()

        conf_matrix_ext = confusion_matrix(
            Y_test_ext, predictions_ext, labels=selected_classes
        )
        array_ext = np.array(conf_matrix_ext)

        plot_heatmap(
            normalized_array=array_ext,
            classes=best_model.classes_,
            conf_matrix=conf_matrix_ext,
            name=f"{combination_name} external test set {name}",
            Y=Y_test_ext,
            combination=combination_name,
        )

        plot_multiclass_roc(
            Y_test_ext,
            probabilities_ext,
            n_classes=len(selected_classes),
            figsize=(6, 5),
            name=f"{combination_name} external test set {name}",
            classes=best_model.classes_,
            combination=combination_name,
        )

        result[f"{name}_Conf_Matrix_ext"] = conf_matrix_ext
        result[f"{name}_Pred_Test_ext"] = predictions_ext
        result[f"{name}_Proba_Test_ext"] = probabilities_ext
        result[f"{name}_ACC_ext"] = np.round(
            accuracy_score(Y_test_ext, predictions_ext), 2
        )
        result[f"{name}_F1_ext"] = np.round(
            f1_score(Y_test_ext, predictions_ext, average="macro"), 2
        )
        result[f"{name}_PREC_ext"] = np.round(
            precision_score(Y_test_ext, predictions_ext, average="macro"), 2
        )
        result[f"{name}_REC_ext"] = np.round(
            recall_score(Y_test_ext, predictions_ext, average="macro"), 2
        )
        result[f"{name}_AUC_macro_ovo_ext"] = np.round(
            roc_auc_score(
                Y_test_ext,
                probabilities_ext[:, 1],
                average="macro",
                multi_class="ovo",
                labels=sorted(selected_classes),
            ),
            2,
        )
        result[f"{name}_log_loss_ext"] = log_loss(
            Y_test_ext, probabilities_ext, labels=sorted(selected_classes)
        )
        result[f"{name}_Conf_Matrix_ext"] = conf_matrix_ext

    results.append(result)

results_df = pd.DataFrame(results).T
results_df.to_excel(os.path.join(RESULT_PATH, "results_Correct_Oversampling.xlsx"))

#### "Incorrect" Approach

In [ ]:
results = []
N_FEATURES = "NUMBER_OF_FEATURES_TO_SELECT"

for combination_name, train_data, test_data in dataset_combinations:
    # select columns for X and Y
    X_train = train_data.drop("Primary", axis=1, inplace=False).astype(float)
    Y_train = train_data["Primary"]

    X_test = test_data.drop("Primary", axis=1, inplace=False).astype(float)
    Y_test = test_data["Primary"]

    # Oversampling incorrectly performed before the dataset split
    SMOTE_Sampler = SMOTE(random_state=RANDOM_SEED)
    X_train_SMOTE, Y_train_SMOTE = SMOTE_Sampler.fit_resample(X_train, Y_train)

    ROS = RandomOverSampler(random_state=RANDOM_SEED)
    X_train_ROS, Y_train_ROS = ROS.fit_resample(X_train, Y_train)

    # Plot external test set distribution
    plot_distribution(Y_test, "External Test Set Wrong Oversampling", combination_name)

    dataframes = [
        (X_train_SMOTE, Y_train_SMOTE, "SMOTE Wrong Oversampling"),
        (X_train_ROS, Y_train_ROS, "ROS Wrong Oversampling"),
    ]

    result = {}
    result["Combination_Name"] = combination_name

    for X, Y, name in dataframes:
        train_inds, test_inds = next(
            StratifiedGroupKFold(n_splits=5).split(X, Y, groups=X.index)
        )

        X_train_int = X.iloc[train_inds]
        X_test_int = X.iloc[test_inds]
        Y_train_int = Y.iloc[train_inds]
        Y_test_int = Y.iloc[test_inds]

        X_test_ext = X_test
        Y_test_ext = Y_test

        scaler = StandardScaler()
        X_train_int = pd.DataFrame(
            scaler.fit_transform(X_train_int), columns=X_train_int.columns
        )
        X_test_int = pd.DataFrame(
            scaler.transform(X_test_int), columns=X_test_int.columns
        )
        X_test_ext = pd.DataFrame(
            scaler.transform(X_test_ext), columns=X_test_ext.columns
        )

        features = LASSO_selection(X_train_int, Y_train_int, N_FEATURES)
        X_train_int = X_train_int.loc[:, features]
        X_test_int = X_test_int.loc[:, features]
        X_test_ext = X_test_ext.loc[:, features]

        plot_distribution(Y_train_int, f"Internal Train Set {name}", combination_name)

        plot_distribution(Y_test_int, f"Internal Test Set {name}", combination_name)

        plot_pca(
            X_train_int,
            Y_train_int,
            selected_classes,
            f"Internal Train Set {name}",
            combination_name,
            kernel_type="rbf",
            gamma_type=1,
            alpha_type=0.1,
        )

        cv_inner = StratifiedGroupKFold(n_splits=5)
        model = RandomForestClassifier(random_state=RANDOM_SEED)

        space = {
            "n_estimators": [10, 100, 500, 1000],
            "max_features": ["sqrt", "auto", 2, 4, 6],
        }

        search = GridSearchCV(model, space, scoring="f1_macro", cv=cv_inner, refit=True)

        res = search.fit(X_train_int, Y_train_int, groups=X_train_int.index)
        best_model = res.best_estimator_

        best_model.fit(X_train_int, Y_train_int)

        # Calculate permutation score to test vs. random labels
        score, perm_scores, pvalue = permutation_test_score(
            best_model, X, Y, n_permutations=100, n_jobs=4, scoring="f1_macro"
        )
 
        # Feature Importance
        sorted_idx = best_model.feature_importances_.argsort()
        plt.barh(
            X_train_int.columns[sorted_idx],
            best_model.feature_importances_[sorted_idx],
            color="#083471",
        )
        plt.xlabel(f"{name} Feature Importance")
        plt.savefig(
            os.path.join(RESULT_PATH, f"Feature_Importance_{combination_name}_{name}.tiff"),
            dpi=300,
            bbox_inches="tight",
        )
        plt.show()
        plt.close()

        predictions_int = pd.Series(
            pd.Categorical(best_model.predict(X_test_int), categories=selected_classes)
        )
        probabilities_int = best_model.predict_proba(X_test_int)
        ytest_dummies_int = pd.get_dummies(Y_test_int, drop_first=False).to_numpy()
        Y_test_int = pd.Series(
            pd.Categorical(Y_test_int, categories=selected_classes),
            index=Y_test_int.index,
        )

        conf_matrix_int = confusion_matrix(
            Y_test_int, predictions_int, labels=selected_classes
        )
        array_int = np.array(conf_matrix_int)

        plot_heatmap(
            normalized_array=array_int,
            classes=best_model.classes_,
            conf_matrix=conf_matrix_int,
            name=f"{combination_name} Internal Test Set {name}",
            Y=Y_test_int,
            combination=combination_name,
        )

        plot_multiclass_roc(
            Y_test_int,
            probabilities_int,
            n_classes=len(selected_classes),
            figsize=(6, 5),
            name=f"{combination_name} Internal Test Set {name}",
            classes=best_model.classes_,
            combination=combination_name,
        )
        result[f"{name}_Length_Train"] = np.shape(Y_train_int)[0]
        result[f"{name}_Length_Test_int"] = np.shape(Y_test_int)[0]
        result[f"{name}_Length_Test_ext"] = np.shape(Y_test_ext)[0]
        result[f"{name}_Cross_Validation_Score"] = np.round(search.best_score_, 2)
        result[f"{name}_Permutation_pvalue"] = np.round(pvalue, 4)
        result[f"{name}_Hyperparameters"] = search.best_params_
        result[f"{name}_Features_Number"] = len(features)
        result[f"{name}_Features"] = features
        result[f"{name}_Model"] = "Random Forest Classifier"
        result[f"{name}_Pred_Test_int"] = predictions_int
        result[f"{name}_Proba_Test_int"] = probabilities_int
        result[f"{name}_ACC_int"] = np.round(
            accuracy_score(Y_test_int, predictions_int), 2
        )
        result[f"{name}_F1_int"] = np.round(
            f1_score(Y_test_int, predictions_int, average="macro"), 2
        )
        result[f"{name}_PREC_int"] = np.round(
            precision_score(Y_test_int, predictions_int, average="macro"), 2
        )
        result[f"{name}_REC_int"] = np.round(
            recall_score(Y_test_int, predictions_int, average="macro"), 2
        )
        result[f"{name}_AUC_macro_ovo_int"] = np.round(
            roc_auc_score(
                Y_test_int,
                probabilities_int,
                average="macro",
                multi_class="ovo",
                labels=sorted(selected_classes),
            ),
            2,
        )
        result[f"{name}_log_loss_int"] = log_loss(
            Y_test_int, probabilities_int, labels=sorted(selected_classes)
        )

        predictions_ext = best_model.predict(X_test_ext)
        probabilities_ext = best_model.predict_proba(X_test_ext)
        ytest_dummies_ext = pd.get_dummies(Y_test_ext, drop_first=False).to_numpy()

        conf_matrix_ext = confusion_matrix(
            Y_test_ext, predictions_ext, labels=selected_classes
        )
        array_ext = np.array(conf_matrix_ext)

        plot_heatmap(
            normalized_array=array_ext,
            classes=best_model.classes_,
            conf_matrix=conf_matrix_ext,
            name=f"{combination_name} external test set {name}",
            Y=Y_test_ext,
            combination=combination_name,
        )

        plot_multiclass_roc(
            Y_test_ext,
            probabilities_ext,
            n_classes=5,
            figsize=(6, 5),
            name=f"{combination_name} external test set {name}",
            classes=best_model.classes_,
            combination=combination_name,
        )

        result[f"{name}_Conf_Matrix_ext"] = conf_matrix_ext
        result[f"{name}_Pred_Test_ext"] = predictions_ext
        result[f"{name}_Proba_Test_ext"] = probabilities_ext
        result[f"{name}_ACC_ext"] = np.round(
            accuracy_score(Y_test_ext, predictions_ext), 2
        )
        result[f"{name}_F1_ext"] = np.round(
            f1_score(Y_test_ext, predictions_ext, average="macro"), 2
        )
        result[f"{name}_PREC_ext"] = np.round(
            precision_score(Y_test_ext, predictions_ext, average="macro"), 2
        )
        result[f"{name}_REC_ext"] = np.round(
            recall_score(Y_test_ext, predictions_ext, average="macro"), 2
        )
        result[f"{name}_AUC_macro_ovo_ext"] = np.round(
            roc_auc_score(
                Y_test_ext,
                probabilities_ext,
                average="macro",
                multi_class="ovo",
                labels=sorted(selected_classes),
            ),
            2,
        )
        result[f"{name}_log_loss_ext"] = log_loss(
            Y_test_ext, probabilities_ext, labels=sorted(selected_classes)
        )
        result[f"{name}_Conf_Matrix_ext"] = conf_matrix_ext

    results.append(result)

results_df = pd.DataFrame(results).T
results_df.to_excel(
    os.path.join(RESULT_PATH, "results_Wrong_Oversampling_combinations.xlsx")
)

#### Final Evaluation incl. Confidence Intervals

In [ ]:
N_FEATURES = "NUMBER_OF_FEATURES_TO_SELECT"

combination_name = "Train = Local+UCSF, Test = STANFORD"
train_data = pd.concat([data_train, data_test_2])
test_data = data_test_1

X_train = train_data.drop("Primary", axis=1, inplace=False).astype(float)
Y_train = train_data["Primary"]

X_test = test_data.drop("Primary", axis=1, inplace=False).astype(float)
Y_test = test_data["Primary"]

train_inds, test_inds = next(
    StratifiedGroupKFold(n_splits=5).split(X_train, Y_train, groups=X_train.index)
)

X_train_int = X_train.iloc[train_inds]
X_test_int = X_train.iloc[test_inds]
Y_train_int = Y_train.iloc[train_inds]
Y_test_int = Y_train.iloc[test_inds]

X_test_ext = X_test.copy()
Y_test_ext = Y_test.copy()

scaler = StandardScaler()
X_train_int = pd.DataFrame(
    scaler.fit_transform(X_train_int), columns=X_train_int.columns
)
X_test_int = pd.DataFrame(scaler.transform(X_test_int), columns=X_test_int.columns)
X_test_ext = pd.DataFrame(scaler.transform(X_test_ext), columns=X_test_ext.columns)

features = LASSO_selection(X_train_int, Y_train_int, N_FEATURES)
X_train_int = X_train_int.loc[:, features]
X_test_int = X_test_int.loc[:, features]
X_test_ext = X_test_ext.loc[:, features]

# Choose Oversampling Strategy
"""SMOTE_Sampler = SMOTE(random_state=RANDOM_SEED, k_neighbors=4)
X_train_int, Y_train_int = SMOTE_Sampler.fit_resample(
   X_train_int, Y_train_int
)"""

ROS = RandomOverSampler(random_state=RANDOM_SEED)
X_train_int, Y_train_int = ROS.fit_resample(X_train_int, Y_train_int)

model = RandomForestClassifier(
    max_features=2,
    n_estimators=500,
    random_state=RANDOM_SEED,
)

model.fit(X_train_int, Y_train_int)

test_data = [
    ("internal test data", X_test_int, Y_test_int),
    ("external test data", X_test_ext, Y_test_ext),
]

for test_name, X_test, y_test in test_data:
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)

    n_iterations = 1000
    labels = np.unique(y_test)
    n_classes = len(labels)

    original_metrics = {
        "accuracy": {},
        "precision": {},
        "recall": {},
        "f1": {},
        "roc_auc": {},
        "accuracy_macro": accuracy_score(y_test, y_pred),
        "precision_macro": precision_score(
            y_test, y_pred, average="macro", zero_division=0
        ),
        "recall_macro": recall_score(y_test, y_pred, average="macro", zero_division=0),
        "f1_macro": f1_score(y_test, y_pred, average="macro", zero_division=0),
        "roc_auc_macro": roc_auc_score(
            y_test, y_prob, average="macro", multi_class="ovo"
        ),
    }

    label_to_index = {label: index for index, label in enumerate(labels)}
    for label in labels:
        is_label = y_test == label
        if np.any(is_label):
            index = label_to_index[label]
            original_metrics["accuracy"][label] = accuracy_score(
                y_test[is_label], y_pred[is_label]
            )
            original_metrics["precision"][label] = precision_score(
                y_test, y_pred, labels=[label], average="macro", zero_division=0
            )
            original_metrics["recall"][label] = recall_score(
                y_test, y_pred, labels=[label], average="macro", zero_division=0
            )
            original_metrics["f1"][label] = f1_score(
                y_test, y_pred, labels=[label], average="macro", zero_division=0
            )
            original_metrics["roc_auc"][label] = roc_auc_score(
                (y_test == label), y_prob[:, index]
            )

        else:
            # Handle cases where a label might not be present in the test set
            original_metrics["accuracy"][label] = "N/A"
            original_metrics["precision"][label] = "N/A"
            original_metrics["recall"][label] = "N/A"
            original_metrics["f1"][label] = "N/A"
            original_metrics["roc_auc"][label] = "N/A"

    bootstrap_metrics = {
        "accuracy": {label: [] for label in labels},
        "precision": {label: [] for label in labels},
        "recall": {label: [] for label in labels},
        "f1": {label: [] for label in labels},
        "roc_auc": {label: [] for label in labels},
        "accuracy_macro": [],
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "roc_auc_macro": [],
    }

    for i in tqdm(range(n_iterations)):
        bootstrap_indices = resample(
            np.arange(len(y_test)), n_samples=len(y_test), random_state=i
        )
        y_true_sample = y_test[bootstrap_indices]
        y_pred_sample = y_pred[bootstrap_indices]
        y_prob_sample = y_prob[bootstrap_indices]
        y_true_sample_binarized = y_true_sample

        # Calculate metrics per label and macro
        for label in labels:
            is_label = y_true_sample == label
            if np.any(is_label):
                bootstrap_metrics["accuracy"][label].append(
                    accuracy_score(y_true_sample[is_label], y_pred_sample[is_label])
                )
                bootstrap_metrics["precision"][label].append(
                    precision_score(
                        y_true_sample,
                        y_pred_sample,
                        labels=[label],
                        average="macro",
                        zero_division=0,
                    )
                )
                bootstrap_metrics["recall"][label].append(
                    recall_score(
                        y_true_sample,
                        y_pred_sample,
                        labels=[label],
                        average="macro",
                        zero_division=0,
                    )
                )
                bootstrap_metrics["f1"][label].append(
                    f1_score(
                        y_true_sample,
                        y_pred_sample,
                        labels=[label],
                        average="macro",
                        zero_division=0,
                    )
                )
                bootstrap_metrics["roc_auc"][label].append(
                    roc_auc_score(
                        (y_true_sample == label),
                        y_prob_sample[:, labels == label],
                        average="macro",
                    )
                )

        bootstrap_metrics["accuracy_macro"].append(
            accuracy_score(y_true_sample, y_pred_sample)
        )
        bootstrap_metrics["precision_macro"].append(
            precision_score(
                y_true_sample, y_pred_sample, average="macro", zero_division=0
            )
        )
        bootstrap_metrics["recall_macro"].append(
            recall_score(y_true_sample, y_pred_sample, average="macro", zero_division=0)
        )
        bootstrap_metrics["f1_macro"].append(
            f1_score(y_true_sample, y_pred_sample, average="macro", zero_division=0)
        )

        try:
            bootstrap_metrics["roc_auc_macro"].append(
                roc_auc_score(
                    y_true_sample_binarized,
                    y_prob_sample,
                    average="macro",
                    multi_class="ovo",
                )
            )
        except ValueError as e:
            print(e)
            bootstrap_metrics["roc_auc"][label].append(np.nan)

    alpha = 0.95
    confidence_intervals = {}
    for metric, values in bootstrap_metrics.items():
        if metric in bootstrap_metrics.keys() - {
            "accuracy_macro",
            "precision_macro",
            "recall_macro",
            "f1_macro",
            "roc_auc_macro",
        }:  # Label-based metrics
            confidence_intervals[metric] = {}
            for label, scores in values.items():
                lower = np.nanpercentile(scores, (1 - alpha) / 2 * 100)
                upper = np.nanpercentile(scores, (alpha + (1 - alpha) / 2) * 100)
                confidence_intervals[metric][label] = (lower, upper)
        else:
            lower = np.nanpercentile(values, (1 - alpha) / 2 * 100)
            upper = np.nanpercentile(values, (alpha + (1 - alpha) / 2) * 100)
            confidence_intervals[metric] = (lower, upper)

    result_df = pd.DataFrame(columns=list(original_metrics.keys()))

    for metric, values in original_metrics.items():
        if metric in ["accuracy", "precision", "recall", "f1", "roc_auc"]:
            for label, original_value in values.items():
                ci = confidence_intervals[metric].get(label, ("N/A", "N/A"))
                result_df.loc[
                    label, metric
                ] = f"{original_value:.2f} [{ci[0]:.2f}, {ci[1]:.2f}]"
        else:
            ci = confidence_intervals[metric]
            result_df.loc[
                "All_Labels", metric
            ] = f"{values:.2f} [{ci[0]:.2f}, {ci[1]:.2f}]"

    result_df.to_excel(RESULT_PATH, "Final_Evaluation.xlsx")